# Listening to the audio
This code enables you to listen to a sample sine wave audio.

In [93]:
%matplotlib inline
import re
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
import numpy
import torch

In [3]:
SAMPLE_RATE = 22050

In [4]:
T = 4.0    # seconds
t = np.linspace(0, T, int(T*SAMPLE_RATE), endpoint=False) # time variable
freq = 440
signal = np.sin(2*np.pi*freq*t)
ipd.Audio(signal, rate=SAMPLE_RATE)

# Creating spectrograms
Spectrograms provide a 2D feature space to analyze signals in the joint time-frequency domain.

This code generates the spectrogram files for use with the CNN example. It makes files for non-noised data!

In [ ]:
# Just some re-shaping and dimension finding
N = 1
print "N:",N
train = signal[np.newaxis,:]
print "Train shape",train.shape
N_train = train.shape[0]
NUM_SAMPLES = train.shape[1]-1

X_train = train[:,:-1]
y_train = train[:,-1]
y_train = y_train.reshape(N_train,1)

In [ ]:
# JUST SOME FOURIER TRANSFORM PARAMETERS
BINS_OCTAVE = 12*2
N_OCTAVES = 7
NUM_BINS = BINS_OCTAVE * N_OCTAVES

In [ ]:
# Given a wav time series, makes a mel spectrogram
# which is a short-time fourier transform with
# frequencies on the mel (log) scale.
def mel_spec(y):
    Q = librosa.cqt(y=y, sr=SAMPLE_RATE, bins_per_octave=BINS_OCTAVE,n_bins=NUM_BINS)
    Q_db = librosa.amplitude_to_db(Q,ref=np.max)
    return Q_db

In [ ]:
# This means that the spectrograms are 168 rows (frequencies)
# By 173 columns (time frames)
song = X_train[0]
test_spec = mel_spec(song)
print test_spec.shape
FEATS = test_spec.shape[0]
FRAMES = test_spec.shape[1]

In [ ]:
tmp_train = np.zeros((N_train,FEATS,FRAMES))
for i in range(N_train):
    tmp_train[i,:,:] = mel_spec(X_train[i])
#np.save('Data/xtrain_spec', tmp_train)

In [ ]:
librosa.display.specshow(tmp_train[0], y_axis='hz')

# Training

In [20]:
from __future__ import division
from keras.layers import Convolution1D, MaxPool1D, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.models import Sequential, load_model
from keras.utils import np_utils

In [71]:
def train_generator(fname='train.csv', step_size=5):
    for step in range(6325//step_size):
        df = pd.read_csv(fname, header=None, skiprows=step*step_size, nrows=step_size).values
        X_train, y_train = df[:,:-1], df[:,-1]
        X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
        y_train = np_utils.to_categorical(y_train,num_classes=10)
        yield (X_train, y_train)
            
def test_generator(fname='test.csv', step_size=5):
    for step in range(1000//step_size):
        df = pd.read_csv(fname, header=None, skiprows=step*step_size, nrows=step_size).values
        labels, X_test = df[:,0], df[:,1:]
        yield (labels, X_test)

In [90]:
# build model
model = Sequential()
model.add(Convolution1D(100, 20, strides=10, activation='relu', input_shape=(88200,1)))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Convolution1D(25, 20, strides=10, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Convolution1D(10, 10, strides=6, activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [91]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_45 (Conv1D)           (None, 8819, 100)         2100      
_________________________________________________________________
batch_normalization_34 (Batc (None, 8819, 100)         400       
_________________________________________________________________
dropout_26 (Dropout)         (None, 8819, 100)         0         
_________________________________________________________________
conv1d_46 (Conv1D)           (None, 880, 25)           50025     
_________________________________________________________________
batch_normalization_35 (Batc (None, 880, 25)           100       
_________________________________________________________________
dropout_27 (Dropout)         (None, 880, 25)           0         
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 146, 10)           2510      
__________

In [92]:
for epoch in range(10):
    for X_train, y_train in train_generator():
        model.fit(X_train, y_train, epochs=1, verbose=0, validation_split=0)

InvalidArgumentError: Node 'batch_normalization_34/cond/Switch': Unknown input node 'batch_normalization_1/keras_learning_phase'

# Predict

In [ ]:
preds, labels = [], []
for labels_curr, X_test in test_generator():
    preds_curr = np.argmax(model.predict(X_test), axis=1)
    labels += list(labels_curr)
    preds += list(preds_curr)

In [43]:
def write_to_file(filename, preds, labels):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for l,p in zip(labels,preds):
            f.write(str(l)+','+str(p)+'\n')

In [ ]:
write_to_file('out.csv', preds, labels)